In [1]:
!conda install --yes --quiet pymongo

import pymongo

from pymongo import MongoClient



# All requested packages already installed.
# packages in environment at /opt/conda:
#
pymongo                   3.4.0                    py36_0    defaults


In [72]:
#make sure to run Mongo from command line sshUbuntu@IP
client = MongoClient('52.38.12.228', 27016)

In [74]:
# create dbs
client.projec4_database

db_ref = client.project4_database
# confirm dbs
client.database_names()

Database(MongoClient(host=['52.38.12.228:27016'], document_class=dict, tz_aware=False, connect=True), 'projec4_database')

In [101]:
# create collections
ML_coll_ref = db_ref.machine_learning_collection
BS_coll_ref = db_ref.business_software_collections
#confirmn collections
db_ref.collection_names()

['machine_learning_collection', 'business_software_collections']

In [71]:
# import article retrieval and cleaning libraries
import re
import requests
import pandas as pd
import numpy as np
import time

In [36]:
# make requests and store to dictionary based on whether its page or category
def category_pages(category):
    pages_dict={}
    subcat_dict={}
    r = requests.get('http://en.wikipedia.org/w/api.php?action=query&\
                list=categorymembers&cmtitle=Category:{}&format=json&cmlimit=max'.format(category))
    
    ml_category_ls = r.json()['query']['categorymembers']
    
    # call pages_v_subcat function
    pages,subcategories = pages_v_subcat(pages_dict,subcat_dict,ml_category_ls)
    
    return pages, subcategories
  

In [37]:
# determine if returned page is category or subcategory
def pages_v_subcat(page_dict,subcat_dict,ml_ls):
    for dictionary in ml_ls:
        # ns !=14 indicates a page
        if dictionary['ns'] != 14:
            page_dict[dictionary['title']] = dictionary['pageid']
        # ns ==14 indicates category
        elif dictionary['ns'] == 14:
            subcat_dict[dictionary['title']] = dictionary['pageid']
            
    return page_dict,subcat_dict

In [38]:
# collect all page_ids for specified number of layers
def collect_page_ids(category,layers=3):
    layers = layers
    pages,subcats = category_pages(category)
    while layers > 0:
        try:
            pages_next_layer = {}
            subcat_next_layer = {}

            for key in subcats.keys():
                p,s = category_pages(key[9:])

                pages_next_layer[key[9:]] = p

                subcat_next_layer.update(s)

            subcats = subcat_next_layer

            for page in pages_next_layer.values():
                for key,page2 in page.items():
                    pages[key]=page2
        except:
            pass
        layers -=1
    return pages

In [39]:
# removes noise from article text
def clean_output(string):
    clean_string = re.sub('<[^<]+?>', '',string)
    clean_string2 = re.sub('\n','',clean_string)
    clean_string3 = re.sub("'",'',clean_string2)
#     clean_ls = element for element in clean_string.split('\n') if element !=''
    return clean_string3

In [40]:
def article_text_to_list (page_dictionary, master_list):
    
    for page_title,page_id in page_dictionary.items():
        
        page_results ={}
        
        page_request = requests.get('http://en.wikipedia.org/w/api.php?\
        action=query&prop=extracts&format=json&pageids={}'.format(page_id)) 
        
        page_text = page_request.json()['query']['pages']['{}'.format(page_id)]['extract']
        
        clean_page_text = clean_output(page_text)
        
        page_results = {'page_title':page_title,
                       'page_id':page_id,
                       'page_text':clean_page_text}
        
        master_list.append(page_results)
        
        

In [96]:
# returns all article text and stores to Mongo Collections
def article_text_to_mongo (page_dictionary, mongo_collection):
    
    for page_title,page_id in page_dictionary.items():
        try:
            page_results ={}

            page_request = requests.get('http://en.wikipedia.org/w/api.php?\
            action=query&prop=extracts&format=json&pageids={}'.format(page_id)) 

            page_text = page_request.json()['query']['pages']['{}'.format(page_id)]['extract']

            clean_page_text = clean_output(page_text)

            page_results = {'page_title':page_title,
                           'page_id':page_id,
                           'page_text':clean_page_text}


            mongo_collection.insert_one(page_results)
        except:
            pass
        
        # delays next request
        time.sleep(np.random.randint(2,5))

In [42]:
# pages = collect_page_ids('Machine learning',layers=3)

In [92]:
# Machine learning page count
len(pages)

1117

In [88]:
# article_text_to_mongo(pages,ML_coll_ref)

In [91]:
ML_coll_ref.count()

1110

In [93]:
# collect business sofrtware pages
bs_pages = collect_page_ids('Business software',layers=5)

In [94]:
# Business software page count
len(bs_pages)

8147

In [97]:
article_text_to_mongo(bs_pages,BS_coll_ref)

In [98]:
BS_coll_ref.count()

8146